<a href="https://colab.research.google.com/github/PsorTheDoctor/Sekcja-SI/blob/master/neural_networks/NSL/document_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph regularization: Klasyfikacja dokumentów z użyciem grafów

In [0]:
!pip install tensorflow-gpu==2.0.1

In [2]:
!pip install -q neural-structured-learning

     |████████████████████████████████| 112kB 13.0MB/s 


## Dependecje i importy

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import neural_structured_learning as nsl

import tensorflow as tf

# Reset notebooka
tf.keras.backend.clear_session()

print('Version: ', tf.__version__)
print('Eager mode: ', tf.executing_eagerly())
print('GPU is', 'available' if tf.test.is_gpu_available() else 'NOT AVAILABLE')

Version:  2.0.1
Eager mode:  True
GPU is available


## Pobranie zbioru danych Cora

In [4]:
%%bash
wget --quiet -P /tmp https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz
tar -C /tmp -xvzf /tmp/cora.tgz

cora/
cora/README
cora/cora.content
cora/cora.cites


## Konwersja danych Cora na format NSL

In [5]:
!wget https://raw.githubusercontent.com/tensorflow/neural-structured-learning/master/neural_structured_learning/examples/preprocess/cora/preprocess_cora_dataset.py

!python preprocess_cora_dataset.py \
--input_cora_content=/tmp/cora/cora.content \
--input_cora_graph=/tmp/cora/cora.cites \
--max_nbrs=5 \
--output_train_data=/tmp/cora/train_merged_examples.tfr \
--output_test_data=/tmp/cora/test_examples.tfr

--2020-03-20 23:38:12--  https://raw.githubusercontent.com/tensorflow/neural-structured-learning/master/neural_structured_learning/examples/preprocess/cora/preprocess_cora_dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11419 (11K) [text/plain]
Saving to: ‘preprocess_cora_dataset.py’

preprocess_cora_dat 100%[===================>]  11.15K  --.-KB/s    in 0s      

2020-03-20 23:38:12 (127 MB/s) - ‘preprocess_cora_dataset.py’ saved [11419/11419]

Reading graph file: /tmp/cora/cora.cites...
Done reading 5429 edges from: /tmp/cora/cora.cites (0.01 seconds).
Making all edges bi-directional...
Done (0.01 seconds). Total graph nodes: 2708
Joining seed and neighbor tf.train.Examples with graph edges...
Done creating and writing 2155 merged tf.train.Examples (

## Zmienne globalne

In [0]:
### Doświadczalny zbiór danych
TRAIN_DATA_PATH = '/tmp/cora/train_merged_examples.tfr'
TEST_DATA_PATH = '/tmp/cora/test_examples.tfr'

### Stałe użyte do identyfikacji cech sąsiada na wejściu
NBR_FEATURE_PREFIX = 'NL_nbr_'
NBR_WEIGHT_SUFFIX = '_weight'

## Hiperparametry

In [0]:
class HParams(object):
  '''Hiperparametry użyte do trenowania.'''
  def __init__(self):
    # parametry zbioru danych
    self.num_classes = 10
    self.max_seq_length = 1433
    # parametry neural graph learning
    self.distance_type = nsl.configs.DistanceType.L2
    self.graph_regularization_multiplier = 0.1
    self.num_neighbors = 1
    # architektura modelu
    self.num_fc_units = [50, 50]
    # parametry trenowania
    self.train_epochs = 100
    self.batch_size = 128
    self.dropout_rate = 0.5
    # parametry oceny
    self.eval_steps = None  # Wszystkie instancje w zbiorze testowym są oceniane.

HPARAMS = HParams()

## Załadowanie danych treningowych i testowych

In [0]:
def parse_example(example_proto):

  feature_spec = {
      'words': tf.io.FixedLenFeature([HPARAMS.max_seq_length],
                                     tf.int64,
                                     default_value=tf.constant(
                                         0,
                                         dtype=tf.int64,
                                         shape=[HPARAMS.max_seq_length])),
      'label': tf.io.FixedLenFeature((), tf.int64, default_value=-1)
  }

  for i in range(HPARAMS.num_neighbors):
    nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, i, 'words')
    nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, i, NBR_WEIGHT_SUFFIX)
    feature_spec[nbr_feature_key] = tf.io.FixedLenFeature(
        [HPARAMS.max_seq_length],
        tf.int64,
        default_value=tf.constant(
            0, dtype=tf.int64, shape=[HPARAMS.max_seq_length]))
    
    feature_spec[nbr_weight_key] = tf.io.FixedLenFeature(
        [1], tf.float32, default_value=tf.constant([0.0]))
    
  features = tf.io.parse_single_example(example_proto, feature_spec)
  label = features.pop('label')
  return features, label
  
    
def make_dataset(file_path, training=False):

  dataset = tf.data.TFRecordDataset([file_path])
  
  if training:
    dataset = dataset.shuffle(10000)
  
  dataset = dataset.map(parse_example)
  dataset = dataset.batch(HPARAMS.batch_size)
  
  return dataset


train_dataset = make_dataset(TRAIN_DATA_PATH, training=True)
test_dataset = make_dataset(TEST_DATA_PATH)

In [17]:
for feature_batch, label_batch in train_dataset.take(1):
  print('Lista cech:', list(feature_batch.keys()))
  print('Wsad wejść:', feature_batch['words'])
  nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, 0, 'words')
  nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, 0, NBR_WEIGHT_SUFFIX)
  print('Wsad sąsiednich wejść:', feature_batch[nbr_feature_key])
  print('Wsad sąsiednich wag:', 
        tf.reshape(feature_batch[nbr_weight_key], [-1]))
  print('Wsad etykiet:', label_batch)

Lista cech: ['NL_nbr_0_weight', 'NL_nbr_0_words', 'words']
Wsad wejść: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(128, 1433), dtype=int64)
Wsad sąsiednich wejść: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(128, 1433), dtype=int64)
Wsad sąsiednich wag: tf.Tensor(
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.], shape=(128,), dtype=float32)
Wsad etykiet: tf.Tensor(
[1 3 1 6 1 2 1 2 2 2 4 5 3 2 2 5 3 5 2 2 2 3 4 2 0 2 6 3 5 3 6 4 3 2 2 1 2
 2 1 6 1 1 2 0 0 1 2 0 2 

In [18]:
for feature_batch, label_batch in test_dataset.take(1):
  print('Lista cech:', list(feature_batch.keys()))
  print('Wsad wejść:', feature_batch['words'])
  nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, 0, 'words')
  nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, 0, NBR_WEIGHT_SUFFIX)
  print('Wsad sąsiednich wejść:', feature_batch[nbr_feature_key])
  print('Wsad sąsiednich wag:', 
        tf.reshape(feature_batch[nbr_weight_key], [-1]))
  print('Wsad etykiet:', label_batch)

Lista cech: ['NL_nbr_0_weight', 'NL_nbr_0_words', 'words']
Wsad wejść: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(128, 1433), dtype=int64)
Wsad sąsiednich wejść: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(128, 1433), dtype=int64)
Wsad sąsiednich wag: tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(128,), dtype=float32)
Wsad etykiet: tf.Tensor(
[5 2 2 2 1 2 6 3 2 3 6 1 3 6 4 4 2 3 3 0 2 0 5 2 1 0 6 3 6 4 2 2 3 0 4 2 2
 2 2 3 2 2 2 0 2 2 2 2 4 

## Budowa modelu
### Model sekwencyjny

In [0]:
def make_mlp_sequential_model(hparams):
  '''Tworzy sekwencyjny perceptron wielowarstwowy.'''
  model = tf.keras.Sequential()
  model.add(
      tf.keras.layers.InputLayer(
          input_shape=(hparams.max_seq_length,), name='words'))
  # Wejście jest już zakodowane w one-hot w formacie integer. 
  # Tutaj przerzucamy je do formatu zmiennoprzecinkowego.
  model.add(
      tf.keras.layers.Lambda(lambda x: tf.keras.backend.cast(x, tf.float32)))
  for num_units in hparams.num_fc_units:
    model.add(tf.keras.layers.Dense(num_units, activation='relu'))
    model.add(tf.keras.layers.Dropout(hparams.dropout_rate))
  model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
  return model

### Model funkcjonalny

In [0]:
def make_mlp_functional_model(hparams):
  '''Tworzy funkcjonalny bazowany na API perceptron wielowarstwowy.'''
  inputs = tf.keras.Input(
      shape=(hparams.max_seq_length,), dtype='int64', name='words')
  
  # Wejście jest już zakodowane w one-hot w formacie integer. 
  # Tutaj przerzucamy je do formatu zmiennoprzecinkowego.
  cur_layer = tf.keras.layers.Lambda(
      lambda x: tf.keras.backend.cast(x, tf.float32))(inputs)

  for num_units in hparams.num_fc_units:
    cur_layer = tf.keras.layers.Dense(num_units, activation='relu')(cur_layer)
    cur_layer = tf.keras.layers.Dropout(hparams.dropout_rate)(cur_layer)

  outputs = tf.keras.layers.Dense(
      hparams.num_classes, activation='softmax')(cur_layer)

  model = tf.keras.Model(inputs, outputs=outputs)
  return model

## Tworzenie modelu bazowego

In [28]:
# Tworzy model bazowy MLP z użyciem funkcjonalnego API.
# Możemy również stworzyć model sekwencyjny używając
# funkcji make_mlp_sequential_model() zdefiniowanej powyżej.
base_model_tag, base_model = 'FUNCTIONAL', make_mlp_functional_model(HPARAMS)
base_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
words (InputLayer)           [(None, 1433)]            0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 1433)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 50)                71700     
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                510 

## Trenowanie modelu MLP

In [29]:
# Kompilacja i trening
base_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
base_model.fit(train_dataset, epochs=HPARAMS.train_epochs, verbose=1)

Epoch 1/100
17/17 [==============================] - 2s 121ms/step - loss: 2.2394 - accuracy: 0.1893
Epoch 2/100
17/17 [==============================] - 0s 11ms/step - loss: 2.0852 - accuracy: 0.2872
Epoch 3/100
17/17 [==============================] - 0s 10ms/step - loss: 1.9035 - accuracy: 0.3341
Epoch 4/100
17/17 [==============================] - 0s 10ms/step - loss: 1.7467 - accuracy: 0.3708
Epoch 5/100
17/17 [==============================] - 0s 10ms/step - loss: 1.5925 - accuracy: 0.4329
Epoch 6/100
17/17 [==============================] - 0s 10ms/step - loss: 1.4380 - accuracy: 0.5077
Epoch 7/100
17/17 [==============================] - 0s 10ms/step - loss: 1.2648 - accuracy: 0.5731
Epoch 8/100
17/17 [==============================] - 0s 10ms/step - loss: 1.1451 - accuracy: 0.6093
Epoch 9/100
17/17 [==============================] - 0s 11ms/step - loss: 1.0304 - accuracy: 0.6585
Epoch 10/100
17/17 [==============================] - 0s 10ms/step - loss: 0.9264 - accuracy: 0.699

## Ocena modelu MLP

In [0]:
# Funkcja pomocnicza do wyświetlenia oceny metryki.
def print_metrics(model_desc, eval_metrics):

  print('\n')
  print('Ocena dokładności dla ', model_desc, ': ', eval_metrics['accuracy'])
  print('Ocena straty dla ', model_desc, ': ', eval_metrics['loss'])
  if 'graph_loss' in eval_metrics:
    print('Ocena straty grafu dla: ', model_desc, ': ', eval_metrics['graph_loss'])

In [33]:
eval_results = dict(
    zip(base_model.metrics_names,
        base_model.evaluate(test_dataset, steps=HPARAMS.eval_steps)))
print_metrics('Model bazowy MLP', eval_results)

5/5 [==============================] - 0s 16ms/step - loss: 1.2421 - accuracy: 0.7884


Ocena dokładności dla  Model bazowy MLP :  0.78842676
Ocena straty dla  Model bazowy MLP :  1.242060649394989


## Trenowanie modelu MLP z regularyzacją grafu

In [0]:
# Budowa nowego bazowego modelu MLP.
base_reg_model_tag, base_reg_model = 'FUNCTIONAL', make_mlp_functional_model(HPARAMS)

In [43]:
graph_reg_config = nsl.configs.make_graph_reg_config(
    max_neighbors = HPARAMS.num_neighbors,
    multiplier = HPARAMS.graph_regularization_multiplier,
    distance_type = HPARAMS.distance_type,
    sum_over_axis = -1)
graph_reg_model = nsl.keras.GraphRegularization(base_reg_model,
                                                graph_reg_config)
graph_reg_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
graph_reg_model.fit(train_dataset, epochs=HPARAMS.train_epochs, verbose=1)

Epoch 1/100


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


17/17 [==============================] - 2s 103ms/step - loss: 2.2285 - accuracy: 0.1689 - graph_loss: 0.0062
Epoch 2/100
17/17 [==============================] - 0s 13ms/step - loss: 2.0685 - accuracy: 0.2724 - graph_loss: 0.0157
Epoch 3/100
17/17 [==============================] - 0s 13ms/step - loss: 1.9114 - accuracy: 0.3104 - graph_loss: 0.0378
Epoch 4/100
17/17 [==============================] - 0s 11ms/step - loss: 1.7506 - accuracy: 0.3610 - graph_loss: 0.0642
Epoch 5/100
17/17 [==============================] - 0s 11ms/step - loss: 1.6132 - accuracy: 0.4255 - graph_loss: 0.0863
Epoch 6/100
17/17 [==============================] - 0s 15ms/step - loss: 1.4572 - accuracy: 0.4854 - graph_loss: 0.1143
Epoch 7/100
17/17 [==============================] - 0s 13ms/step - loss: 1.3511 - accuracy: 0.5258 - graph_loss: 0.1450
Epoch 8/100
17/17 [==============================] - 0s 12ms/step - loss: 1.2085 - accuracy: 0.5763 - graph_loss: 0.1823
Epoch 9/100
17/17 [========================

In [44]:
eval_results = dict(
    zip(graph_reg_model.metrics_names,
        graph_reg_model.evaluate(test_dataset, steps=HPARAMS.eval_steps)))
print_metrics('MLP + regularyzacja grafu', eval_results)

5/5 [==============================] - 0s 95ms/step - loss: 1.1085 - accuracy: 0.8119 - graph_loss: 0.0000e+00


Ocena dokładności dla  MLP + regularyzacja grafu :  0.8119349
Ocena straty dla  MLP + regularyzacja grafu :  1.1085497200489045
Ocena straty grafu dla:  MLP + regularyzacja grafu :  0.0
